In [1]:
!pip install transformers peft datasets torch accelerate wandb matplotlib pandas numpy evaluate rouge-score nltk langchain langchain-community langchain-huggingface pypdf docx2txt chromadb

Часть 1: Подготовка данных

In [3]:
# Импорт библиотек
import json
from func_to_call import parse_all_data

# Парсинг данных
train_data = parse_all_data('train_set.json')
val_data = parse_all_data('val_set.json')

# Форматирование данных
def format_dataset(data):
    formatted = []
    for item in data:
        contexts = "\n".join([ctx['text'] for ctx in item['contexts']])
        winner = item['winner']
        if winner == 'Saiga':
            output = item['saiga_answer']
        elif winner == 'GigaChat':
            output = item['giga_answer']
        else:
            continue  # Пропускаем случаи вроде "Оба плохо" или "Оба хорошо"
        formatted.append({
            "input": f"Ты - система информационного поиска НИУ ВШЭ. Создай краткий и информативный ответ на вопрос, используя только информацию из контекста. Используй непредвзятый и журналистский тон. Завершай ответ 'FINAL ANSWER'.\nВопрос: {item['user_question']}\nКонтекст: {contexts}",
            "output": f"{output}\nFINAL ANSWER"
        })
    return formatted

train_formatted = format_dataset(train_data)
val_formatted = format_dataset(val_data)

# Сохранение для удобства
with open('train_formatted.json', 'w', encoding='utf-8') as f:
    json.dump(train_formatted, f, ensure_ascii=False)
with open('val_formatted.json', 'w', encoding='utf-8') as f:
    json.dump(val_formatted, f, ensure_ascii=False)

print(f"Train samples: {len(train_formatted)}, Val samples: {len(val_formatted)}")

Train samples: 354, Val samples: 89


Часть 2: Загрузка и настройка модели

In [16]:
# Импорт библиотек
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import json

# Загрузка модели
model_name = "google/mt5-base"  # Меняем на mT5
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Загрузка данных
with open('train_formatted.json', 'r', encoding='utf-8') as f:
    train_formatted = json.load(f)
with open('val_formatted.json', 'r', encoding='utf-8') as f:
    val_formatted = json.load(f)

train_dataset = Dataset.from_list(train_formatted)
val_dataset = Dataset.from_list(val_formatted)

def preprocess_function(examples):
    inputs = examples["input"]
    targets = examples["output"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized = train_dataset.map(preprocess_function, batched=True)
val_tokenized = val_dataset.map(preprocess_function, batched=True)

# Проверка токенизации
print("Decoded input:", tokenizer.decode(train_tokenized[0]["input_ids"], skip_special_tokens=True)[:200])
print("Decoded output:", tokenizer.decode(train_tokenized[0]["labels"], skip_special_tokens=True)[:200])

# Настройка LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
)
model = get_peft_model(model, lora_config)

print("Model loaded and configured with LoRA")

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/354 [00:00<?, ? examples/s]

Map:   0%|          | 0/89 [00:00<?, ? examples/s]

Decoded input: Ты - система информационного поиска НИУ ВШЭ. Создай краткий и информативный ответ на вопрос, используя только информацию из контекста. Используй непредвзятый и журналистский тон. Завершай ответ 'FINAL
Decoded output: Подробное изучение науки, проведение исследований, участие в конкурсах способствует развитию научных знаний и навыков, необходимых для успешного выполнения выпускной квалификационной работы. Эти мероп
Model loaded and configured with LoRA


In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")

# Простой русский текст
text = "Ты - система информационного поиска НИУ ВШЭ. Вопрос: Что такое наука?"
encoded = tokenizer(text, max_length=512, truncation=True, padding="max_length")["input_ids"]
decoded = tokenizer.decode(encoded, skip_special_tokens=True)

print("Original text:", text)
print("Decoded text:", decoded)

Original text: Ты - система информационного поиска НИУ ВШЭ. Вопрос: Что такое наука?
Decoded text: Ты - система информационного поиска НИУ ВШЭ. Вопрос: Что такое наука?


Часть 3: Обучение модели

In [19]:
# Импорт библиотек
from transformers import Trainer, TrainingArguments
import wandb

# Настройка обучения
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
)

trainer.train()

model.save_pretrained("./best_model")
tokenizer.save_pretrained("./best_model")

print("Training completed and model saved")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


KeyboardInterrupt: 

Часть 4: Интеграция с RAG и генерация ответов

In [ ]:
# Импорт библиотек
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from rag_simple import load_chroma, generate_answer
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Создание pipeline для модели
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
llm = HuggingFacePipeline(pipeline=pipe)

# Загрузка векторной базы данных
embeddings_e5 = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
vectordb = load_chroma("./chroma", embeddings_e5)

# Тест на одном вопросе
question = val_data[0]["user_question"]
answer, source_docs = generate_answer(question, llm, vectordb)
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"Source documents: {len(source_docs)} found")

Часть 5: Валидация и подсчет метрик

In [ ]:
# Импорт библиотек
from metrics import ValidatorSimple
import pandas as pd

# Генерация ответов для всего val_set
val_answers = []
for item in val_data:
    answer, _ = generate_answer(item["user_question"], llm, vectordb)
    val_answers.append(answer)

# Формирование датафрейма для метрик
val_df = pd.DataFrame({
    "question": [item["user_question"] for item in val_data],
    "answer": val_answers,
    "ground_truth": [item["saiga_answer"] if item["winner"] == "Saiga" else item["giga_answer"] for item in val_data],
    "contexts": [[ctx["text"] for ctx in item["contexts"]] for item in val_data]
})

# Подсчет метрик
vs = ValidatorSimple(neural=True)  # Нужен GPU для neural метрик
results = vs.validate_rag(val_df)
print("Validation results:", results)

# Логирование в WandB
wandb.log(results)

Часть 6: Визуализация и финализация

In [ ]:
# Импорт библиотек
import matplotlib.pyplot as plt

# Визуализация лосса
history = trainer.state.log_history
train_loss = [h["loss"] for h in history if "loss" in h]
eval_loss = [h["eval_loss"] for h in history if "eval_loss" in h]
epochs_train = range(1, len(train_loss) + 1)
epochs_eval = range(1, len(eval_loss) + 1)

plt.plot(epochs_train, train_loss, label="Train Loss")
plt.plot(epochs_eval, eval_loss, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.savefig("loss_plot.png")
plt.show()

# Завершение WandB
wandb.finish()

print("Visualization saved as 'loss_plot.png'")